In [61]:
import json
import numpy as np
import keras
import keras.preprocessing.text as kpt
from keras.preprocessing.text import Tokenizer
from keras.models import model_from_json
from keras.preprocessing.sequence import pad_sequences
import json
import keras
import pandas as pd
import keras.preprocessing.text as kpt
from keras.preprocessing.text import Tokenizer
import numpy as np
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense, LSTM, SimpleRNN, Dropout, Activation, Embedding
from datetime import datetime
from tensorflow.keras.callbacks import EarlyStopping

import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix

import json
from keras.models import model_from_json

# Import libraries
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
import nltk 
# nltk.download()
import string
import re
%matplotlib inline
pd.set_option('display.max_colwidth', 100)

import string
import re
from keras.preprocessing.text import Tokenizer
from nltk.stem import WordNetLemmatizer 

In [62]:
# we're still going to use a Tokenizer here, but we don't need to fit it
tokenizer = Tokenizer(num_words=10000)
# for human-friendly printing
labels = ['fake', 'real']

# read in our saved dictionary
with open('./Dictionary_Models/glove_models_dictionary.json', 'r') as dictionary_file:
    dictionary = json.load(dictionary_file)

In [63]:
def convert_text_to_index_array(text):
    words = kpt.text_to_word_sequence(text)
#     words = pad_sequences(words, padding='post', maxlen=23)
#     print(text)
    wordIndices = []
    for word in words:
        if word in dictionary:
            wordIndices.append(dictionary[word])
    return wordIndices

In [64]:
json_file = open('./NN_Models/Glove_BLSTM_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()

model = model_from_json(loaded_model_json)

model.load_weights('./NN_Models/Glove_BLSTM_model.h5')


In [65]:
tweetText = 'there is a fire at melville mountain'

tweetText = convert_text_to_index_array(tweetText)

input = pad_sequences([tweetText], padding='post', maxlen=100)
    
pred = model.predict_generator(input)

def rounding(results):
    '''Results needs to be rounded to 0 or 1 for fake or real, respectively'''
    if results < 0.5:
        return 0
    else:
        return 1
    
predictions_final = [rounding(x) for x in pred]

predictions_final

[1]

# Connecting to Tweepy

In [66]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy import API
from tweepy import Cursor


import sys  
sys.path.insert(0, './Python_Models/')
import credentials
# from DataCleaning import clean_text, tokenization, remove_stopwords, listToString, lementization

In [67]:
#punctutation removal
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

# Tockenization
def tokenization(text):
    text = re.split('\W+', text)
    return text

# stopword removal 
stopword = nltk.corpus.stopwords.words('english')

def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text


# return to string
def listToString(s):     
    # initialize an empty string 
    str1 = " " 
    # return string   
    return (str1.join(s)) 

# Lematizing the words
lemmatizer = WordNetLemmatizer()

def lementization(text):
    text = lemmatizer.lemmatize(text)
    return text


In [68]:
class TwitterAuthenticator():
    def authenticate_twitter_app(self):
        auth = OAuthHandler(credentials.CONSUMER_KEY, credentials.CONSUMER_SECRET)
        auth.set_access_token(credentials.ACCESS_TOKEN, credentials.ACCESS_TOKEN_SECRET) 
        return auth       

In [69]:
class TweeterStreamer():
    def __init__(self):
        self.twitter_authenticator = TwitterAuthenticator()
      
    def stream_tweets(self, fetched_tweet_file, hash_tag_list):
        #handles twitter authentication and twitter streaming API
        listener = TwitterListener(fetched_tweet_file) 
        auth = self.twitter_authenticator.authenticate_twitter_app()       
        stream = Stream(auth, listener)
        
        stream.filter(track=hash_tag_list)
         

In [70]:
def DataCleaningFunction(text):
    text = clean_text(text)
    text = tokenization(text.lower())
    text = remove_stopwords(text)
    text = listToString(text)
    text = lementization(text.lower())
    return text


In [71]:
def rounding(results):
    '''Results needs to be rounded to 0 or 1 for fake or real, respectively'''
    if results < 0.5:
        return 0
    else:
        return 1

In [74]:
class TwitterListener(StreamListener):  
    def __init__(self, fetched_tweets_filename):
        self.fetched_tweets_filename = fetched_tweets_filename
    
    def on_data(self, data):
        try:
            tweetText = json.loads(data)['text']
            myvar = tweetText
            tweetText = DataCleaningFunction(tweetText)
            testArr = convert_text_to_index_array(tweetText)
            input = pad_sequences([testArr], padding='post', maxlen=100)
            
            pred = model.predict_generator(input)

            predictions_final = [rounding(x) for x in pred]
            
            print('\n',myvar)
            
            print('****',labels[predictions_final[0]], 'Tweet')
            
            # print("%s \n the %s sentiment; %f%% confidence" % (tweetText ,labels[np.argmax(pred)], pred[0][np.argmax(pred)] * 100))
      
            with open(self.fetched_tweets_filename, 'a') as tf:
                tf.write(data)
            return True
        except BaseException as e:
            print("Error on data: %s" % str(e))
        return True
    
    def on_error(self, status):
        if status == 420:
            return False
        print(status)
        
        

In [ ]:
if __name__ == "__main__":
    mylist =['hazard','fire','ablaze','accident','aftershock','ambulance']   
    fetched_tweets_filename = "text.txt"
    
    tweeter_streamer = TweeterStreamer()
    tweeter_streamer.stream_tweets(fetched_tweets_filename,mylist)
    


 RT @IfeAdebayo: How do you lose a COLONEL in battle and everywhere is silent like nothing happened. By now the whole of the Armed Forces sh…
**** fake Tweet

 RT @LindseyGrahamSC: Being lectured by Democrats about how to handle judicial nominations is like an arsonist advising the Fire Department.
**** real Tweet

 RT @FadingzZ: リクエストよりEchoesのメイです
リクエストありがとうございました！！😭

Request for Mae from Fire Emblem: Echoes.
Thank you so much for requesting! !😭 https:…
**** fake Tweet

 RT @CaseyDancing: casey frey dancing to september by earth, wind &amp; fire https://t.co/PAodGhH5Pl
**** real Tweet

 RT @DPPonline: 在文明世界中，動不動就以戰爭作為恐嚇手段的，叫做「恐怖組織」。環球時報的威脅，證明了兩岸最大的差異不是經濟或軍事，而是文明程度。
**** fake Tweet

 sentenced toはよく使われますね。
hangは吊るすっていう意味があるので、sentenced to hangはつまり死刑です。
**** real Tweet

 poxa só pq eu ia fzr isso
**** fake Tweet

 RT @DeliciouslySavv: 5 Things You Should Keep In Mind After A Car Accident https://t.co/kRAP1mRf4L
**** fake Tweet

 RT @RadioChinar: As the joint team encircled the suspecte


 RT @kumabozu111: ちび、助けたい！

あきらめない！

あなたの優しさが命を救います！

いいね もいいけど
#拡散希望
#リツイート

#注目
#猫好き
#猫のいる幸せ

ご支援・ご協力
よろしくお願いします！
**** fake Tweet

 RT @louiseteaches: Teach me to abide in love
The way I have learnt
to abide in fear

Teach me to lay aside
this scorched glove
&amp; still play…
**** fake Tweet

 RT @nonchel_: @kenshin110708 オンラインライブまであと3日しかないけど覚えられるように頑張るね🥺
早く実際のライブでコール叫びたい🥰🥰
曲に活気が出ていいね❗💓💓
粗品さん笑ったᜦ ᜦ ᜦ fireの部分ほんとおもしろかった😂直弥くんと目合わせながら…
**** fake Tweet

 @adekola1990 Was on the line for one U17 game Sunday. First since being injured last November and my both my hamstr… https://t.co/XvuHAhxp7K
**** fake Tweet

 RT @marseillebiblio: Découvrez notre sélection du #Mardi Skye a un rêve : devenir chasseuse de foudre⚡⚡, mais elle y a renoncé depuis l’acc…
**** fake Tweet

 RT @sixsevenkilla: I just made a fire ass new video 😉 more previews soon : https://t.co/YQUVgKBzTC https://t.co/QymEt374Iq
**** fake Tweet

 RT @AskAnshul: Now, it's revealed that in Delhi riots, some rioters had first checke


 RT @ErtzFoundation: Some women fear the fire, some women simply become it 🔥  #MondayMotivation https://t.co/6FOA7ZLIBC
**** fake Tweet

 @TREASUREunion @treasuremembers PLAYING WIGH FIRE

@treasuremembers
 #TREASURE #트레저
**** fake Tweet

 RT @tsuki_yuuka_: 遊んでいる月です✨
皆様のご支援、ご協力のおかげでここまで元気になることができました！！クラウドファンディングの締め切りが本日11時59分までです。まだ目標に達していません😢最後にご支援、ご協力のほどお願い致します🙇‍♀️
クラウドファンディ…
**** fake Tweet

 RT @jeffphilips1: Atiku's goons hv spent the whole day defending their unarmed robber perpetual candidate.

iT Is jUsT a bAnKiNg rOuTiNe cH…
**** fake Tweet

 RT @JesseKellyDC: I too have a proposal. We nominate the most fire-breathing conservative we can find and put them on the court. Then, afte…
**** fake Tweet

 RT @CaseyDancing: casey frey dancing to september by earth, wind &amp; fire https://t.co/PAodGhH5Pl
**** real Tweet

 What I like about this &amp; all of the quote tweets.. we all might have some disagreements, but we all agree that best… https://t.co/L5zaoY2gYQ
**** fake Tweet

 R


 You’re welcome bro they are SO fire
**** fake Tweet

 RT @toadspinning: september - Earth, Wind and Fire https://t.co/IegATQWNzc
**** real Tweet

 RT @DWFireControl: 09:40 @SbourneFireStn @DWFRSWestbourne @RedhillFireStn attend light fitting fire in building - https://t.co/pWNGl7JB2c
**** real Tweet

 @drihanna_dm I f*** with your tat ahh😯♥😭 And your bikini brand FIRE...the lines must be booming, I pray your busine… https://t.co/BqIYWyIVWP
**** fake Tweet

 RT @53kyCreation12: 😍That eyes says All🔥
70 runs of 30 balls👌
5️⃣ #sixes 5️⃣ #Fours are jst fire woks 🎆🎆

💥💥One of the Crucial knock by @YU…
**** fake Tweet

 RT @btsyoutubedata: Top 11 @BTS_twt music videos on YouTube

1. DNA — 1,089 Billion
2. Boy With Luv — 950.8M
3. Fake Love — 773.8M
4. IDOL…
**** fake Tweet

 RT @TRTWorldNow: #BREAKING: Pakistani court sentences two political activists to death over 2012 Karachi garment factory fire that killed m…
**** real Tweet

 Well! Q5! We are predicting hail, floods, fire warnings and 


 RT @jasondogwood: You lost all credibility when you starting getting blackmailed by Putin for having sex with that underage boy in Russia.…
**** fake Tweet

 RT @prettywetgirl: Had a leaky accident in public this weekend...please don’t make fun of me, I already had to make it all the way home in…
**** fake Tweet

 RT @EricaRN4USA: President @realDonaldTrump is on fire 🔥 tonight in Ohio! Get out and vote my fellow Ohioans!!! Vote early and take friends…
**** real Tweet

 H=R
R=W
W&gt;Fi
R+W=WFo=W
W&gt;Fi
Cali= 0*R + 0*SW
Cali = Fi
**** fake Tweet

 RT @taetaemybunny: #WAYV : WE ARE SO FIRE 🔥 
https://t.co/g9pJRKGBiE
**** fake Tweet

 RT @KASHINNISMF: Wassup twitter. Ion tweet much but i recently got in a life threatening accident. I broke my spine, lost my new car, lost…
**** fake Tweet

 RT @s0rbetero: My sister is a nurse in the US. Kaya siya nandun dahil sobrang baba ng pasahod ng mga nars sa Pinas. Hindi nila kailangan ng…
**** fake Tweet

 RT @JasariFromMars: Best fire sign- Leo 

...


 RT @OnI6SgnWZKCCWLM: FIP闘病中テオちゃん🆘😿
クラウドファンディング現在4%と非常に厳しい状況です😭🆘
FIPのお薬は、目が飛び出る程高額で、皆様お力添えお願いしたいです😭
500円からご支援可能です😭
ご支援ご協力お願いします😭🆘
#拡散RT希望
#ク…
**** fake Tweet

 RT @advmonikaarora: Rioters had checked the ownership of vehicles through e-Vaahan portal before setting them on fire https://t.co/s0H3TTVK…
**** real Tweet

 RT @AskAnshul: Now, it's revealed that in Delhi riots, some rioters had first checked the e-Vaahan website before setting vehicles on fire.…
**** real Tweet

 RT @UpenRathod8: Horny macho bot!! Enjoy my candy 🍭!! Let’s play baby!! Do watch full video on https://t.co/o8jPTci6yG !! Hotness On!! Fire…
**** fake Tweet

 RT @KASHINNISMF: Wassup twitter. Ion tweet much but i recently got in a life threatening accident. I broke my spine, lost my new car, lost…
**** fake Tweet

 UPDATED - Fire crews say the house on Ave. A was vacant, story updated with new video: 
 https://t.co/HyBssuSGtl
**** real Tweet

 RT @georgelamps: The same thing we dragged Sarri and Mourinho and Conte f


 From there while the fire features were Popping I was there thinking Nukata had replaced Hey &amp; Cartoon as my favori… https://t.co/XQ3zkaoGm2
**** fake Tweet

 @GayRepublicSwag Regional convince store chain will fire employees if they catch them allowing patrons in without m… https://t.co/GE5nSKPziq
**** real Tweet

 House On Fire - D078, D083, E069, E057, L075, E075, A069, E083, L069, SF057, M083 - Candlewood Dr - 05:42 - https://t.co/HTsxiFGjrz
**** fake Tweet

 RT @AndrewLoeschner: 9 times out of 10 the trending topics on Twitter are a dumpster fire.
**** fake Tweet

 incrível que o T3ddy é mais atencioso que meu pai. 🙃
**** fake Tweet

 House On Fire - L034, E030, SF030, E012, M034, D031, E034, E039, L031, D034 - Jensen Dr - 05:48 - https://t.co/8jyY2tCJlx
**** fake Tweet

 Building Fire - E040 - Gulf Fwy Ib - Griggs Entr - 05:36 - https://t.co/E0J6a9ejir
**** real Tweet

 Building Fire - D026, E023, E026, E025, L020, E020, SF024, L026, D020, M040 - Gulf Fwy Ib - Griggs Entr -


 RT @GridironSchol91: Kickoff Week 4 of #CollegeFootball with "College Fantasy Football Waiver Wire Gemstones" on @Fantrax.
Eric, @CFFroton,…
**** fake Tweet

 RT @blackxagustd: 'Cause I, I, I'm in the stars tonight
So watch me bring the fire and set the night alight 🎶💜 https://t.co/ugUOFDDLHW
**** fake Tweet

 RT @Mariee337: PLEASE keep your eyes open for this man. He murdered a child here in Reno and set him on fire. He’s on the run and deserves…
**** fake Tweet

 ♈️
**** fake Tweet

 RT @Ma3RouteCounty: 13:44 Accident along mtito Voi highway. Anybody with details of the car owner to inform friends and family. He's in a c…
**** real Tweet

 RT @Goddessjisooyaa: YG should fire their editor and hire some badass blinks who can edit a poster like a professional
**** fake Tweet

 RT @KASHINNISMF: Wassup twitter. Ion tweet much but i recently got in a life threatening accident. I broke my spine, lost my new car, lost…
**** fake Tweet

 RT @Yawsikigh: But the cowardly, unbelieving, abomina


 yaaaaAASSSSS DESERVEEEE my ambulance ost #LINETHPeoplesChoiceAwards
**** fake Tweet

 @imVkohli ❤️ @actorvijay

🔥@charufan1 🔥@MuthuVirat4🔥 🔥@Haneesh_virat🔥@DarkSoul18_🔥 @itzNisha7 🔥@KaushalSKV🔥… https://t.co/1yvB5eCIle
**** fake Tweet

 RT @SarahJa28484793: @AleshaOfficial @AshleyBanjo You insulted millions of Brits with your ‘reverse racism’...’entitlement ‘ mantra...and n…
**** fake Tweet

 Tesla's safety record isn't the best. And they are not the only culprit who are rushing out new products and lettin… https://t.co/jTUSOlPAVC
**** fake Tweet

 That petrol tranker and taxi  accident killing 12 people including a pregnant woman yoh 😭💔
**** real Tweet

 残り3時間！

ビバ★開運したい方はご覧ください！

開運ポッドキャストで日本を明るく元気にしたい！ - クラウドファンディングCAMPFIRE https://t.co/zWRrc6bmsH #夢見る人をはじめる人に @campfirejpから
**** fake Tweet

 @EASPORTSFIFA EA, It's time for you to fire your Bartomeu. The game has turned into an arcade game for smartphones.… https://t.co/6768PdowN0
**** fake Tweet

 RT @DebraMessing: OK, America. I 


 RT @Fuyinari: Big Fire Cat. https://t.co/YprTMjaIzO
**** fake Tweet

 RT @KASHINNISMF: Wassup twitter. Ion tweet much but i recently got in a life threatening accident. I broke my spine, lost my new car, lost…
**** fake Tweet

 RT @PASTELKREEPA: fire and air sign friendship: https://t.co/wk2MjXZA0N
**** real Tweet

 RT @JesseKellyDC: I too have a proposal. We nominate the most fire-breathing conservative we can find and put them on the court. Then, afte…
**** fake Tweet

 RT @t3ddyyyyy: bom dia crianças
boa ead pra vcs

so nao pode mutar o professor pra jogar free fire ok
&lt;3
**** fake Tweet

 RT @CaseyDancing: casey frey dancing to september by earth, wind &amp; fire https://t.co/PAodGhH5Pl
**** real Tweet

 The 26-year-old stated he had taken the knife from his co-accused but could not remember holding it to his throat. https://t.co/1ORmhGbI9K
**** fake Tweet

 RT @AsieduMends: The #Yolo track really be fire
**** fake Tweet

 RT @_hellothere95: Fan: "Unni this could be a sensitiv